<a href="https://colab.research.google.com/github/luqmanhaqim/Haqim-repo/blob/main/Report_generator_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
import json 
import pandas as pd 
import openpyxl
from datetime import datetime, timedelta, date
import pytz
from pytz import timezone

def scanTable():
  response = table.scan()
  items = response['Items']
  while 'LastEvaluatedKey' in response:
    print(response['LastEvaluatedKey'])
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response['Items'])

  print(items)
  print(len(items))

  with open("Agent_Trigger_Table_PreProd.json","w") as f:
    data=json.dump(items,f,indent=4)

  records=""
  with open("Agent_Trigger_Table_PreProd.json","r") as f:
    records=json.load(f)
    # print(f.read())
  
  # print(records)
  return records
  print(len(records))

# records=scanTable

def createReportingDf():
  data={
    "ID":[],
    "Policy_Number":[],
    "Entity":[],
    "Call_Answered":[],
    "Stages_Completed":[],
    "Verification":[],
    "Trigger_Timestamp":[],
    "Policy_Received":[],
    "Survey_Rating":[],
  }

  df=pd.DataFrame(data)

  df.columns
  return df

# df=createReportingDf

def extractData(records,df):
  
  wholedict=records #initial dictionary 
  B=0
  botList=[]

  for i in wholedict:
    subDict=wholedict[B]

    if subDict.get('Bot_Comprehensibility') == None:
      botList.append("NA")
    else:
        Bot_Comprehensibility_List=subDict["Bot_Comprehensibility"]
        lastStage=max(Bot_Comprehensibility_List[len(Bot_Comprehensibility_List)-1].keys())
        botList.append(lastStage)

    B+=1
  df["Stages_Completed"]=botList

#GET ID
  B=0
  id=[]
  for i in wholedict:
    subDict=wholedict[B]
    if subDict.get('ID') == None:
      id.append("NA")
    else:
      id.append(subDict["ID"])


    B+=1
  df["ID"]=id

#GET POLICY NUMBER

  B=0
  policy_number=[]
  for i in wholedict:
    subDict=wholedict[B]
    policy_number.append(subDict["Policy_Number"])

    B+=1
  df["Policy_Number"]=policy_number


#Get Entity
  entity=[]
  B=0

  for i in wholedict:
    
    i= i['Policy_Number']
    if i.startswith("TR"):
        entity.append("FTA")
    elif i.startswith("LR"):
      entity.append("FTA")
    else:
        entity.append("LIA")
    B+=1

  df["Entity"]=entity

#GET CALL STATUS 
  B=0
  call_status=[]
  for i in wholedict:
    subDict=wholedict[B]
    if subDict.get('Call_Answered') == None:
      call_status.append("NA")
    else:
      call_status.append(subDict["Call_Answered"])

    B+=1
  df["Call_Answered"]=call_status
#GET VERIFICATION
  b=0
  verification=[]
  for i in wholedict:
    if wholedict[b].get('Verification') == None:
      verification.append("NA")
    else:
      verification.append(wholedict[b]["Verification"])

    b+=1
  df["Verification"]=verification
#GET TIMESTAMP
  B=0
  timestamp=[]
  for i in wholedict:
    subDict=wholedict[B]
    if subDict.get('Trigger_Timestamp') == None:
      timestamp.append("NA")
    else:
      timestampa= datetime.strptime(subDict["Trigger_Timestamp"],'%Y-%m-%dT%H:%M:%S+08:00')
      strf= datetime.strftime(timestampa,'%Y-%m-%d %H:%M:%S')
      timestamp.append(strf)
      B+=1

  df["Trigger_Timestamp"]=timestamp

 

#GEt Policy Received Status 
  B=0
  policy_received=[]
  for i in wholedict:
    subDict=wholedict[B]
    if subDict.get('Policy_Received') == None:
        policy_received.append("Did not reach the stage")
    else:
      policy_received.append(subDict["Policy_Received"])

    B+=1
  df["Policy_Received"]=policy_received

#GEt Survey Rating 

  B=0
  Survey_Rating=[]
  for i in records:
    subDict=records[B]

    if subDict.get('Survey_Rating') == None:
      Survey_Rating.append("Not applicable")
    else:
        Survey_Rating_List=subDict["Survey_Rating"]
        Survey_Rating.append(Survey_Rating_List)

    B+=1

  df["Survey_Rating"]=Survey_Rating
  

  
  df.to_csv('Agent_pre_prod_ExtractedData.csv', index=False)
  return df

def getData_CDT():
  dynamodb = boto3.resource("dynamodb",aws_access_key_id=access_key,aws_secret_access_key=secret_access_key,region_name='ap-southeast-1')
  table_CDT= dynamodb.Table("Customer_Data_Table")
  response = table_CDT.scan(ProjectionExpression="Policy_Number, Policyholder_Phone_Number")
  items = response['Items']
  while 'LastEvaluatedKey' in response:
    response = table_CDT.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response['Items'])

  df = pd.DataFrame(columns=["Policy_Number", "Policyholder_Phone_Number"])

  for item in items:
    df = df.append({
        "Policy_Number": item['Policy_Number'],
        "Policyholder_Phone_Number": item['Policyholder_Phone_Number']
    }, ignore_index=True)

  df.to_csv("Customer_Data.csv", index=False)
  data_CDT=df
  return data_CDT


#merge CDT with extracted
def filter_data_by_date_range(left_df, right_df, start_date, end_date):
    # Merge the two data frames on the "Policy_Number" column
    merged_df = pd.merge(left_df, right_df, on="Policy_Number", how="left")

    # Reorder the columns so that "Policyholder_Phone_Number" is next to "Policy_Number"
    cols = merged_df.columns.tolist()
    cols.remove("Policyholder_Phone_Number")
    cols = cols[:cols.index("Policy_Number")+1] + ["Policyholder_Phone_Number"] + cols[cols.index("Policy_Number")+1:]
    merged_df = merged_df[cols]

    # Convert the start and end dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

    # Filter the "Trigger_timestamp" column by specified date range
    Finalized_df = merged_df[(merged_df["Trigger_Timestamp"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date() >= start_date)) &
                             (merged_df["Trigger_Timestamp"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date() <= end_date))]

    # Save the filtered data frame to a new CSV file
    return Finalized_df


def Unanswered_Calls(df):
  unanswered_T= df[(df["Call_Answered"]=='True') & (df["Stages_Completed"]=="T.1")]
  Calls_False= df[(df["Call_Answered"]=='False')]
  Calls_To_Be_Scheduled=pd.concat([unanswered_T,Calls_False])
  Calls_To_Be_Scheduled=Calls_To_Be_Scheduled[["Policy_Number","Policyholder_Phone_Number"]]
  return Calls_To_Be_Scheduled

def currentTime():
  now = datetime.now(timezone("Singapore"))
  print(now)
  result1= now
  strf= datetime.strftime(result1,'%Y-%m-%d %I %p')
  print(strf)
  return strf

# def clean_data(data):
#     from numpy import nan
#     nan == nan
#     # Get the unique stages order
#     stages = ["NA",nan, 'T.1', 'F.1', '1.1', '1.2', '1.3', '2.1', '2.2', '2.3', '3.1', '3.2', '4.1', '4.2', '4.3', '5.1', '5.2', '5.3', '5.4', '5.5', '6.1']

#     # Create a dictionary with the stages as keys and their order as values
#     stage_dict = {stages[i]: i for i in range(len(stages))}
#     print(stage_dict)

#     # Create a column with the order of each stage
#     data['stage_order'] = data['Stages_Completed'].apply(lambda x: stage_dict[x])

#     # # Group the data by Policy_Number and keep only the rows with the highest stage_order
#     # data = data.groupby(['Policy_Number'], as_index=False).apply(lambda x: x.loc[x.stage_order.idxmax()])

#     # # Sort data based on ascending Trigger_Timestamp
#     # data.sort_values(by='Trigger_Timestamp', inplace=True, ascending=True)

#     # data=data.drop("stage_order",axis=1)

#     # return data

#     def timestampFormat_date(x):
#       x=datetime.strptime(x, "%d/%m/%Y %H:%M")
#       x=datetime.strftime(x,"%d/%m/%y")
#       return x

#     def timestampFormat_time(x):
#       x=datetime.strptime(x, "%d/%m/%Y %H:%M")
#       x=datetime.strftime(x,"%H:%M")
#       return x

#     data=data.sort_values(by=["Trigger_Timestamp","stage_order"],ascending=False)
#     data=data.drop_duplicates(subset=["Policy_Number","Trigger_Timestamp"],keep="first")

#     data=data.drop(["stage_order",],axis=1)

#     return data
def timestampFormat_date(x):
  #x=datetime.strptime(x, "%d/%m/%Y %H:%M")
  x=datetime.strftime(x,"%d/%m/%y")
  return x

def timestampFormat_time(x):
  #x=datetime.strptime(x, "%d/%m/%Y %H:%M")
  x=datetime.strftime(x,"%H:%M")
  return x

def getHours(x):
    #x=datetime.strptime(x, "%d/%m/%Y %H:%M")
    x=datetime.strftime(x,"%H")
    return x 


def clean_data(x):
  #add relevant columns
  from numpy import nan
  nan == nan
  x["Trigger_Timestamp"]=pd.to_datetime(x["Trigger_Timestamp"])
  stages = ["NA",nan, 'T.1', 'F.1', '1.1', '1.2', '1.3', '2.1', '2.2', '2.3', '3.1', '3.2', '4.1', '4.2', '4.3', '5.1', '5.2', '5.3', '5.4', '5.5', '6.1']
  ranking_dict = {i: stages.index(i) for i in stages}
  x.loc[:, "Dates"] = x["Trigger_Timestamp"].apply(timestampFormat_date)
  x.loc[:, "HANA Call Time"] = x["Trigger_Timestamp"].apply(timestampFormat_time)
  x.loc[:, "Stages_Reached"] = x["Stages_Completed"].apply(lambda x: ranking_dict[x])
  x.loc[:, "Hours of the day"] = x["Trigger_Timestamp"].apply(getHours)

  #sort and drop values 
  x = x.sort_values(by=["Trigger_Timestamp","Stages_Reached"],ascending=False)
  x = x.drop_duplicates(subset=["Policy_Number","Trigger_Timestamp"],keep="first")
  x = x.sort_values(by=["Trigger_Timestamp","Stages_Reached"],ascending=False)
  x = x.drop_duplicates(subset=["Policy_Number","Dates","Hours of the day"],keep="first") 
  # x = x.drop_duplicates(subset=["Policy_Number","Hours of the day"],keep="first")
  print(x)
  #drop irrelevant columns
  x = x.drop(columns=["Stages_Reached","Hours of the day","Dates"],axis=1)
  x=x[['ID', 'Policy_Number','Entity','Policyholder_Phone_Number','HANA Call Time', 'Verification', 'Policy_Received', 'Survey_Rating','Stages_Completed',
       'Call_Answered', 
       'Trigger_Timestamp',
       ]]
  return x

def toExcel(Finalized_df,Calls_To_Be_Scheduled,strf):
    with pd.ExcelWriter(f'Hana_Call_Summary {strf}.xlsx') as writer:
    
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
      Finalized_df.to_excel(writer, sheet_name=f'Call Summary', index=False)
      Calls_To_Be_Scheduled.to_excel(writer, sheet_name=f'Unanswered Calls', index=False)


def getDataPreProd():
  records=scanTable()
  print(len(records))
  df=createReportingDf()
  left_df=extractData(records,df)
  data_CDT= getData_CDT()
  data_CDT=pd.read_csv("/content/Customer_Data.csv")
  df=filter_data_by_date_range(left_df, data_CDT, "2023-02-14","2023-02-17")
  df=clean_data(df)
  Calls_To_Be_Scheduled=Unanswered_Calls(df)
  strf=currentTime()
  toExcel(df,Calls_To_Be_Scheduled,strf)
  return df
